# Implementando uma rede simples (MLP) usando o Keras

## Objetivo

Este notebook implementa um modelo de rede neural simples (MLP) utilizando o Keras para classificação binária com o dataset Adult/Census Income. O objetivo é prever se uma pessoa tem renda acima de US$50K/ano com base em atributos demográficos e profissionais.


## Sobre o Dataset

### Características principais:
- **Nome**: Adult / Census Income (também chamado "Adult" ou "Census Income")
- **Volume**: Aproximadamente 49.000 amostras
- **Features**: 14 atributos (numericos e categóricos) como idade, escolaridade, ocupação, sexo, horas trabalhadas por semana, etc.
- **Tarefa de classificação**: Predizer se a renda anual (label) de uma pessoa é '>50K' (classe 1) ou '<=50K' (classe 0).
- **Desafio**: Dados reais, mistura de variáveis numéricas e categóricas, classes desbalanceadas.


## Carregando e Explorando o Dataset

In [ ]:
import pandas as pd

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain",
    "capital-loss", "hours-per-week", "native-country", "income"
]
df = pd.read_csv(url, header=None, names=column_names, na_values=' ?')

print("Shape:", df.shape)
print(df.dtypes)
print(df["income"].value_counts())


Shape: (32561, 15)
age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object
income
<=50K    24720
>50K      7841
Name: count, dtype: int64


#### Análise inicial
O dataset possui cerca de 32.561 linhas e 15 colunas (14 features + 1 label).
O atributo 'income' é o alvo da classificação, com dois possíveis valores: '>50K' e '<=50K'.
O conjunto apresenta features numéricas (age, hours-per-week, capital-gain/loss, etc.) e categóricas (workclass, education, marital-status, occupation, etc.), além de alguns valores nulos.


## Pré-processamento


In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Download do dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain",
    "capital-loss", "hours-per-week", "native-country", "income"
]
df = pd.read_csv(url, header=None, names=column_names, na_values=' ?')

# Remove linhas com valores ausentes
df = df.dropna()

# Conversão da coluna target para valores binários
df["income"] = df["income"].apply(lambda x: 1 if x.strip() == ">50K" else 0)

# Define as features (X) e label (y)
X = df.drop("income", axis=1)
y = df["income"]

# One-hot encoding para variáveis categóricas
X = pd.get_dummies(X)

# Normalização das variáveis numéricas
num_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

from sklearn.model_selection import train_test_split

# Separação em treino e teste (estratificada)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Train shape: (24129, 104)
Test shape: (6033, 104)


In [8]:
import numpy as np

# Convertendo para float32, requisito para TensorFlow
X_train = np.array(X_train).astype(np.float32)
X_test = np.array(X_test).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)


## Construindo o Modelo MLP no Keras

**Modelo sequencial simples:**
- Uma camada Dense com apenas **1 unidade** (neurônio) e ativação **sigmoid**
- Objetivo: saída entre 0 e 1, para classificação binária

**Funções:**
- **Otimizador Adam**: ajusta os pesos dinamicamente durante o treino.
- **Perda binary_crossentropy**: mede a distância entre predições e rótulos reais em problemas binários.
- **Métricas accuracy e F1**: acurácia mede a proporção de acertos; F1 pondera precisão e recall, fundamental em problemas com desbalanceamento.


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from keras import backend as K
import numpy as np

# Calcular pesos para balancear as classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(enumerate(class_weights))

# Definindo a função personalizada F1 para Keras
def f1_score(y_true, y_pred):
    y_pred = tf.round(y_pred)
    tp = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32))
    fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, tf.float32))
    fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), tf.float32))
    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    recall = tp / (tp + fn + tf.keras.backend.epsilon())
    return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

# Construção do modelo: duas camadas ocultas com ReLU e Dropout para regularização
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compilação do modelo
model.compile(
    optimizer=Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy', f1_score]
)


model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 32)             │         3,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,905 (15.25 KB)

 Trainable params: 3,905 (15.25 KB)

 Non-trainable params: 0 (0.00 B)

## Treinamento do Modelo

In [12]:
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=10,
    validation_split=0.1,
    class_weight=class_weight_dict,
    verbose=1
)


Epoch 1/50
2172/2172 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.7299 - f1_score: 0.2938 - loss: 0.4987 - val_accuracy: 0.8139 - val_f1_score: 0.2862 - val_loss: 0.3874
Epoch 2/50
2172/2172 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7994 - f1_score: 0.2842 - loss: 0.4046 - val_accuracy: 0.8119 - val_f1_score: 0.2889 - val_loss: 0.3830
Epoch 3/50
2172/2172 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8045 - f1_score: 0.2805 - loss: 0.3864 - val_accuracy: 0.8040 - val_f1_score: 0.2922 - val_loss: 0.3871
Epoch 4/50
2172/2172 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8007 - f1_score: 0.2888 - loss: 0.3929 - val_accuracy: 0.8168 - val_f1_score: 0.2871 - val_loss: 0.3627
Epoch 5/50
2172/2172 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.8062 - f1_score: 0.2811 - loss: 0.3826 - val_accuracy: 0.8247 - val_f1_score: 0.2728 - val_loss: 0.3565
Epoch 6/50
2172/2172 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.8091 - f1_score: 0.2778 - loss: 0.3810 - val_accuracy: 0.8185 - val_

## Avaliação no Teste

In [13]:
# Prevendo as probabilidades
y_pred_prob = model.predict(X_test)
# Convertendo para rótulos binários (cutoff 0.5)
y_pred = (y_pred_prob > 0.5).astype(int)

from sklearn.metrics import accuracy_score, f1_score

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Acurácia: {acc:.4f}")
print(f"F1: {f1:.4f}")


189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Acurácia: 0.8066
F1: 0.6828


## Interpretação dos Resultados

## Interpretação dos Resultados

### Desempenho obtido
O modelo simples deve apresentar uma **acurácia entre 80%-85%** e F1 um pouco menor, por conta do desbalanceamento das classes.

- **Acurácia** quantifica a proporção de previsões corretas em relação ao total.
- **F1 Score** pondera precisão e recall, importante em cenários onde as classes são desbalanceadas (há mais exemplos de uma classe do que outra).

### Discussão e Melhorias
Apesar da simplicidade do modelo (apenas 1 neurônio), resultados razoáveis podem ser obtidos graças ao volume e riqueza do dataset. Para melhorar:
- Adicionar camadas ocultas e neurônios aumenta a capacidade de modelagem.
- Regularização (Dropout, L2), ajuste do balanceamento (oversampling/undersampling), e tratamento avançado dos features podem aprimorar os resultados.
- Testes com diferentes funções de ativação, otimização e hiperparâmetros também podem ser investigados.

Assim, este notebook serve como base para experimentações com classificadores binários em dados desafiadores e reais.